In [ ]:
#setup

In [212]:
import sys, os
sys.path.insert(0,os.path.realpath("./fastai/old"))

import fastai
from fastai.text.all import *
#from fastai.plots.all import *

import spacy 

import warnings

from sklearn.metrics import confusion_matrix
import functools

import torch
import pickle

from scipy.stats import zscore

import requests
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import datetime as dt
import matplotlib.dates as mdates
from datetime import datetime, timedelta

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn import model_selection

In [156]:
torch.backends.cudnn.enabled = False

In [218]:
cwd = os.getcwd()
print(cwd)

/Users/tina/Desktop/GitHub/ml_project


In [ ]:
# training and validation

In [220]:
PATH=Path('data/boc/')
CLAS_PATH=Path('data/clas/')
LM_PATH=Path('data/lm/')

In [157]:
CLASSES = ['neg', 'pos', 'unsup', 'meh']

def get_texts(path):
    texts,labels = [],[]
    for idx,label in enumerate(CLASSES):
        with open(str(path/(label+".txt")), "tr") as f:
            for line in f:
                texts.append("xbos xfld 1 " + line.rstrip())
                labels.append(idx)
    return np.array(texts),np.array(labels)

trn_texts,trn_labels = get_texts(PATH/'train')

In [160]:
len(trn_texts)

14921

In [161]:
print(trn_texts[1:5])

['xbos xfld 1 The shutdown will dampen economic growth in the fourth quarter, through the direct effect of furloughed government workers, and, more broadly, through negative effects on confidence.'
 'xbos xfld 1 In contrast, the economies of Brazil, Russia and Turkey are much weaker, partly as a result of political or geopolitical tensions.'
 'xbos xfld 1 In the euro area, real economic activity in the final quarter of 2004 was also weaker than anticipated, and the outlook has been revised down marginally.'
 'xbos xfld 1 Recovery in the advanced economies is still expected to be subdued, since private demand remains weak in most of these countries and fiscal stimulus will start to wind down later this year.']


In [162]:
#shuffling sentences

np.random.seed(np.prod([ord(c) for c in 'deaa']))
trn_idx = np.random.permutation(len(trn_texts))
trn_texts = trn_texts[trn_idx]
trn_labels = trn_labels[trn_idx]

In [163]:
# data split

trn_texts,val_texts = sklearn.model_selection.train_test_split(
    np.concatenate([trn_texts]), test_size=0.1)

In [164]:
len(trn_texts), len(val_texts)

(13428, 1493)

In [ ]:
#format daa + store as CSV 

In [165]:
col_names = ['labels','text']

df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=col_names)

df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)

df_trn = pd.read_csv(LM_PATH/'train.csv', header=None)#, chunksize=chunksize)
df_val = pd.read_csv(LM_PATH/'test.csv', header=None)#, chunksize=chunksize)

In [166]:
df_trn

,0,1
0,0,"xbos xfld 1 Growth in the broad monetary aggregates has moderated as well, mainly reflecting the contribution from narrow money."
1,0,xbos xfld 1 It will then increase modestly to close to 3 per cent over the four quarters of 2006 and through 2007.
2,0,xbos xfld 1 Household credit continues to grow at a pace near its average since 1992 and has picked up in recent months.
3,0,"xbos xfld 1 In line with developments in the core inflation rate, total CPI inflation is also expected to return to the Bank's 2 per cent target in the third quarter of 2011."
4,0,"xbos xfld 1 The output gap, estimated at 1 per cent in the third quarter of 2004, is projected to narrow further during 2006."
...,...,...
13423,0,"xbos xfld 1 Growth in the ""rest of the world"" grouping is expected to increase from about 1 3/4 per cent in 2017 to around 2 3/4 per cent in 2019."
13424,0,"xbos xfld 1 Since reaching a three-year high of approximately 69 cents US at the time of the last Report, the Canadian dollar has continued its sharp appreciation, rising above 75 cents in October."
13425,0,xbos xfld 1 The rise in the Canadian dollar prices received by Canadian commodity producers has been magnified by the depreciation of the Canadian dollar and has been reflected in the marked 9 per cent rise over the past year in the aggregate industrial product price index as well as in the sharp rise in aggregate export prices.
13426,0,"xbos xfld 1 In assessing the outlook for inflation, the Bank must consider various factors, including the external economic environment, the momentum of the Canadian economy and the implications of monetary conditions for the future course of aggregate demand."


In [ ]:
#tokenizating text

In [180]:
import numpy as np
import spacy

# using spacy tokenizer instead of Fastai 
nlp = spacy.load("en_core_web_sm")

def get_tokenized_texts(df, n_lbls=1, tokenizer=nlp):
    labels = df.iloc[:, range(n_lbls)].values.astype(np.int64)
    texts = df.iloc[:, 1].astype(str)
    tok = [[token.text for token in tokenizer(text)] for text in texts]
    return tok, list(labels)

In [181]:
tok_trn, trn_labels = get_tokenized_texts(df_trn)
tok_val, val_labels = get_tokenized_texts(df_val)

In [182]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(10)

[('the', 22335),
 (',', 15700),
 ('in', 14321),
 ('1', 13682),
 ('xbos', 13428),
 ('xfld', 13428),
 ('.', 13424),
 ('of', 11811),
 ('to', 10568),
 ('and', 8602)]

In [188]:
max_vocab = 3000
min_freq = 2

itos = [o for o,c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [189]:
len(itos)

3002

In [199]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
vs = len(itos)

In [191]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (13428,) + inhomogeneous part.

In [198]:
print(tok_trn[1:2])

[['xbos', 'xfld', '1', 'It', 'will', 'then', 'increase', 'modestly', 'to', 'close', 'to', '3', 'per', 'cent', 'over', 'the', 'four', 'quarters', 'of', '2006', 'and', 'through', '2007', '.']]


In [200]:
import numpy as np

# Assume `tok_trn` and `tok_val` are lists of tokenized texts

# Find the maximum length of sequences in both training and validation sets
max_length = max(max(len(seq) for seq in tok_trn), max(len(seq) for seq in tok_val))

# Define the padding token
padding_token = '_pad_'

# Pad sequences in both training and validation sets
tok_trn_padded = [seq + [padding_token] * (max_length - len(seq)) for seq in tok_trn]
tok_val_padded = [seq + [padding_token] * (max_length - len(seq)) for seq in tok_val]

# Convert padded sequences to NumPy arrays
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn_padded])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val_padded])

In [202]:
print(max_length)

98


In [205]:
print(seq)

['xbos', 'xfld', '1', 'CPI', 'inflation', 'has', 'fallen', 'since', 'the', 'beginning', 'of', 'the', 'year', ',', 'reaching', 'an', 'estimated', '1.4', 'per', 'cent', 'on', 'average', 'in', 'the', 'second', 'quarter', ',', 'while', 'two', 'of', 'the', 'Bank', "'s", 'three', 'measures', 'of', 'core', 'inflation', 'have', 'also', 'continued', 'to', 'decline', '.']


In [ ]:
# building LM model

In [207]:
em_sz,nh,nl = 400,1150,3

In [228]:
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

In [229]:
import h5py

In [230]:
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

UnpicklingError: invalid load key, 'v'.

In [234]:
#enc_wgts = to_np(wgts['0.encoder.weight'])
#row_m = enc_wgts.mean(0)

In [235]:
# Open the HDF5 file
with h5py.File(PRE_LM_PATH, 'r') as f:
    # Read the dataset(s) from the HDF5 file
    dataset = f['dataset_name'][:]
    
    # Convert the data to PyTorch tensors
    tensor_data = torch.tensor(dataset)  # or torch.from_numpy(dataset)

# Use the tensor data as needed
print(tensor_data)

OSError: Unable to open file (file signature not found)